This notebook is converts filters in non votable form to votable format.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
0246c5d (Thu Jan 25 17:01:47 2018 +0000) [with local modifications]


In [2]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import seaborn.apionly as sns

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

from astropy.io.votable import parse
from astropy.io.votable import parse_single_table
from astropy.io import ascii
from astropy.io.votable.tree import VOTableFile, Resource, Field
from astropy.io.votable import from_table, writeto

/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [3]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

#filters_folder = "/Users/rs548/GitHub/herschelhelp_python/database_builder/format_changing/"

## Mosaic Ks

In [4]:
mosaic_ks = ascii.read('./format_changing/mosaic_ks.txt')

In [5]:
mosaic_ks

col1,col2
float64,float64
1.9,0.0
1.91,0.005
1.92,0.015
1.93,0.05
1.94,0.16
1.95,0.55
1.96,0.91
1.97,0.95
1.98,0.96


In [6]:
mosaic_ks['col1'].name = 'Wavelength'
# Convert units to angstroms
mosaic_ks['Wavelength'] = mosaic_ks['Wavelength']*1.e4
mosaic_ks['Wavelength'].unit = u.angstrom
mosaic_ks['col2'].name = 'Transmission'
mosaic_ks.meta["Description"] = 'Ks Mosaic'
mosaic_ks.meta["Band"] = 'Ks'
mosaic_ks.meta["Facility"] = 'KPNO'
mosaic_ks.meta["Instrument"] = 'Mosaic'
# Required meta

In [7]:
mosaic_ks

Wavelength,Transmission
Angstrom,
float64,float64
19000.0,0.0
19100.0,0.005
19200.0,0.015
19300.0,0.05
19400.0,0.16
19500.0,0.55
19600.0,0.91
19700.0,0.95


In [8]:
writeto(mosaic_ks, './filters/mosaic_ks.xml')

In [9]:
mosaic_b = ascii.read('./format_changing/mosaic_b.dat')
mosaic_b['col1'].name = 'Wavelength'
mosaic_b['Wavelength'].unit = u.angstrom
mosaic_b['col2'].name = 'Transmission'
#Convert percent to fraction
mosaic_b['Transmission'] = mosaic_b['Transmission']*1.e-2
#mosaic_b
mosaic_b.meta["Description"] = 'Bw Mosaic k1025'
mosaic_b.meta["Band"] = 'Bw'
mosaic_b.meta["Facility"] = 'KPNO'
mosaic_b.meta["Instrument"] = 'Mosaic'

In [10]:
mosaic_b

Wavelength,Transmission
Angstrom,
float64,float64
3200.0,-0.0001
3202.0,-0.00015
3204.0,-5e-05
3206.0,0.0001
3208.0,-0.0001
3210.0,-0.00015
3212.0,-0.0001
3214.0,-0.0002


In [11]:
writeto(mosaic_b, './filters/mosaic_b.xml')

## Suprime narrow

|name in COSMOS2015| name in HELP| Name on download page
    'ia484': 'suprime_ia484',
    'ia527': 'suprime_ia527',
    'ia624': 'suprime_ia624',
    'ia679': 'suprime_ia679',
    'ia738': 'suprime_ia738',
    'ia767': 'suprime_ia767',
    'ib427': 'suprime_ib427',
    'ib464': 'suprime_ib464',
    'ib505': 'suprime_ib505',
    'ib574': 'suprime_ib574',
    'ib709': 'suprime_ib709',
    'ib827': 'suprime_ib827',

In [12]:
suprime_filters = [
    'ia484',
    'ia527',
    'ia624',
    'ia679',
    'ia738',
    'ia767',
    'ib427',
    'ib464',
    'ib505',
    'ib574',
    'ib709',
    'ib827'
]


In [13]:
sup_test = ascii.read('./format_changing/IB427.SuprimeCam.pb')
sup_test[:10].show_in_notebook()
sup_test.meta

OrderedDict([('comments',
              ['filter: IB427',
               'instrument: SuprimeCam',
               'description: Telescope + CCD + filter + atmosphere',
               'reference: None',
               'comment: Filter Passband from S. Sasaki web page, file IB3_07.txt',
               'comment: CCD and Optics transmission derived from the filter curves',
               'comment: at http://indus.astron.s.u-tokyo.ac.jp/works/suprime/filters/',
               'comment: Litterature inband flux at 0 mag and bandwidth computed from bandpass',
               'comment: Prepared by H. Aussel (herve dot aussel at cea dot fr)',
               'url: http://cosmos.phys.sci.ehime-u.ac.jp/~sasaki/NB+IA_FILTER/filter.html',
               'system: ab',
               'primary standard: abstd',
               'primary magnitude: 0.0',
               'bandwidth (xunit): 195.80568365897',
               'inband flux at 0 mag: 1.17279863299192E-09',
               'airmass: 1.2',
          

In [16]:
for f in suprime_filters:
    tab = ascii.read('./format_changing/{}.SuprimeCam.pb'.format(f.upper()))
    tab['file:'].name = 'Wavelength'
    tab['Wavelength'].unit = u.angstrom
    tab['{}.SuprimeCam.pb'.format(f.replace('a','b').upper())].name = 'Transmission'
    tab.meta["Description"] = 'Suprime {}'.format(f)
    tab.meta["Band"] = f
    tab.meta["Facility"] = 'Subaru'
    writeto(tab, './filters/suprime_{}.xml'.format(f.lower()))

In [18]:
suprime_zp = ascii.read('./format_changing/suprime_zp.res')
suprime_zp['col1'].name = 'Wavelength'
suprime_zp['Wavelength'].unit = u.angstrom
suprime_zp['col2'].name = 'Transmission'
#Use over SVO doubles?
#writeto(tab, './filters/suprime_zp.xml')

col1,col2
float64,float64
7914.0,0.000215892080002
7928.3,0.000634934720207
7942.6,0.00113008889163
7956.8,0.00159978317232
7971.1,0.00215826491558
7985.4,0.00280554569585
7999.7,0.0034653847341
8013.9,0.00390642843915
8028.2,0.00440613522022


In [ ]:
suprime_zpp = ascii.read('./format_changing/suprime_zpp.res')
suprime_zpp['col1'].name = 'Wavelength'
suprime_zpp['Wavelength'].unit = u.angstrom
suprime_zpp['col2'].name = 'Transmission'
#Use over SVO doubles?
#writeto(tab, './filters/suprime_zp.xml')